In [1]:
from PIL import Image
import numpy as np
import cv2
import os

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
root="/content/gdrive/Shared with me/Dataset/dataset-resized"

In [5]:
#root = "D:/College Work/Internship/AMU CSS/Dataset/dataset-resized/"

In [6]:
files_papaya = []
# r=root, d=directories, f = files
for r, d, f in os.walk(root + "papaya"):
    for file in f:
        files_papaya.append(file)
files_papaya.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

In [7]:
X = []
Y = []
for image in files_papaya[0:300]:
    img = Image.open(root + 'papaya/' + image)
    img.load()
    data = np.asarray(img)
    data = cv2.cvtColor(data, cv2.COLOR_BGRA2BGR)
    X.append(data)
    Y.append(0)

In [8]:
files_pepper = []
# r=root, d=directories, f = files
for r, d, f in os.walk(root + "pepper"):
    for file in f:
        files_pepper.append(file)
files_pepper.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

In [9]:
for image in files_pepper[0:300]:
    img = Image.open(root + 'pepper/' + image)
    img.load()
    data = np.asarray(img)
    data = cv2.cvtColor(data, cv2.COLOR_BGRA2BGR)
    X.append(data)
    Y.append(1)

In [10]:
for image in files_papaya[300:500]:
    img = Image.open(root + 'papaya/' + image)
    img.load()
    data = np.asarray(img)
    data = cv2.cvtColor(data, cv2.COLOR_BGRA2BGR)
    X.append(data)
    Y.append(0)

for image in files_pepper[300:500]:
    img = Image.open(root + 'pepper/' + image)
    img.load()
    data = np.asarray(img)
    data = cv2.cvtColor(data, cv2.COLOR_BGRA2BGR)
    X.append(data)
    Y.append(1)

In [11]:
X = np.array(X)
Y = np.array(Y)

In [12]:
X.shape

(0,)

In [13]:
from skimage.io import imshow
import matplotlib.pyplot as plt

idx = 300

imshow(np.squeeze(X[idx,:,:,:]))
plt.show()

labels = ['Papaya','Pepper']
print ('This is:',labels[int(Y[idx])])

IndexError: ignored

In [14]:
from tensorflow.keras.layers import Input

# We have 2 inputs, 1 for each picture
left_input = Input((400,400,3))
right_input = Input((400,400,3))

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Conv2D, Flatten, Dense

# We will use 2 instances of 1 network for this task
base_network = Sequential([
    Conv2D(10,3, input_shape=(400,400,3)), Activation('tanh'),
    MaxPooling2D(),
    Conv2D(15,3),
    Activation('tanh'),
    MaxPooling2D(),
    Conv2D(7,2),
    Activation('tanh'),
    MaxPooling2D(),
    Conv2D(7,2),
    Activation('relu'),
    Flatten(),
    Dense(10),
    Activation('sigmoid')
])

In [16]:
processed_a = base_network(left_input)
processed_b = base_network(right_input)

In [17]:
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K

# Getting the L1 Distance between the 2 encodings
L1_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))

# Add the distance function to the network
L1_distance = L1_layer([processed_a, processed_b])

In [18]:
from tensorflow.keras.models import Model

prediction = Dense(1,activation='sigmoid')(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

In [19]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(0.001, decay=2.5e-4)

siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=['accuracy'])

In [20]:
import random

# First let's separate the dataset from 1 matrix to a list of matricies
image_list = np.split(X[:600],600)
label_list = np.split(Y[:600],600)

left_input = []
right_input = []
targets = []

#Number of pairs per image
pairs = 5
#Let's create the new dataset to train on
for i in range(len(label_list)):
    for _ in range(pairs):
        compare_to = i
        while compare_to == i: #Make sure it's not comparing to itself
            compare_to = random.randint(0,599)
        left_input.append(image_list[i])
        right_input.append(image_list[compare_to])
        if label_list[i] == label_list[compare_to]:# They are the same
            targets.append(1)
        else:# Not the same
            targets.append(0)
            
left_input = np.squeeze(np.array(left_input))
right_input = np.squeeze(np.array(right_input))
targets = np.squeeze(np.array(targets))

pepper_img = X[300]
test_left = []
test_right = []
test_targets = []

for i in range(Y.shape[0]-600):
    test_left.append(pepper_img)
    test_right.append(X[i+600])
    test_targets.append(Y[i+600])

test_left = np.squeeze(np.array(test_left))
test_right = np.squeeze(np.array(test_right))
test_targets = np.squeeze(np.array(test_targets))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


IndexError: ignored

In [ ]:
print(len(left_input), len(right_input), len(targets))

In [ ]:
fig = plt.figure(figsize=(8, 8))
#imshow(np.squeeze(left_input[10]))
#plt.show()
inx = int(np.random.rand(1,1)*3000)
fig.add_subplot(2, 2, 1)
plt.imshow(np.squeeze(left_input[inx]))
fig.add_subplot(2, 2, 2)
plt.imshow(np.squeeze(right_input[inx]))
print("Index is:", inx)
s_inx = "SAME" if targets[inx] else "Different"
print(s_inx, targets[inx])

In [ ]:
print(len(test_left), len(test_right), len(test_targets))

In [ ]:
fig = plt.figure(figsize=(8, 8))
#imshow(np.squeeze(left_input[10]))
#plt.show()
dnx = int(np.random.rand(1,1)*400)
fig.add_subplot(2, 2, 1)
plt.imshow(np.squeeze(test_left[dnx]))
fig.add_subplot(2, 2, 2)
plt.imshow(np.squeeze(test_right[dnx]))
print("Index is:", dnx)
s_dnx = "SAME" if test_targets[dnx] else "Different"
print(s_dnx, test_targets[dnx])

In [ ]:
siamese_net.fit([left_input,right_input], targets,
          batch_size = 16,
          epochs=100,
          verbose=1,
          validation_data=([test_left,test_right],test_targets))